# Verkehrsfluss

Die Stadt New York teilt ausführliche Verkehrsinformationen
[hier](https://data.cityofnewyork.us/Transportation/Real-Time-Traffic-Speed-Data/qkm5-nuaq)
und eine Erklärung der Daten befindet sich
[hier](https://data.cityofnewyork.us/api/views/i4gi-tjb9/files/cc7f3b15-58b7-46e3-94e7-4c5753c3a8b8?download=false&filename=metadata_trafficspeeds.pdf).
Diese Verkehrsdaten möchten wir dahingehend untersuchen, ob sich für bestimmte Straßenabschnitte wiederkehrende Muster ergeben.

In [ ]:
# Die folgenden Pakete sind nicht Teil der Standard-Conda-Umgebung.
# Beim ersten Ausführen des Jupyter Notebooks müssen Sie diese also zusätzlich installieren. Falls sie eines der Pakete installieren
# wollen, entfernen Sie die Raute und führen die Zelle aus. Fügen Sie am besten danach die Raute sicherheitshalber wieder hinzu, damit
# beim nächsten Ausführen dieser Zelle nicht erneut versucht wird, die Bibliothek zu installieren.

# %pip install sodapy

# %pip install folium

In [ ]:
import os

import pandas as pd
from pandas.plotting import autocorrelation_plot
import matplotlib.pyplot as plt

from sodapy import Socrata

pd.options.display.max_rows = 10  # zeige maximal 10 Einträge eines DataFrames an

Als Erstes werden die Positionen der Streckenabschnitte geladen.
Eine ausführliche Erklärung, was die Attribute bedeuten, ist [hier](https://dev.socrata.com/foundry/data.cityofnewyork.us/i4gi-tjb9) zu finden.

Allerdings bremst die Stadt New York, wenn es zu viele Anfragen gleichzeitig gibt. Sollte der Prozess zu lange dauern, benötigen Sie einen sogenannten App Token zur Identifikation, der ebenfalls [hier](https://dev.socrata.com/foundry/data.cityofnewyork.us/i4gi-tjb9) beantragt werden kann.

Falls Sie einen App Token erhalten haben, schreiben Sie diesen anstelle des `None` in folgender Zeile:

In [ ]:
app_token = None  # <-- hier den Wert händisch eintragen oder einfach auf None stehen lassen

if app_token is None and os.path.isfile(".app_token.txt"):
    with open(".app_token.txt") as f:
        app_token = f.read()

print("App Token: ", app_token)

Nun können wir den Client initialisieren.

In [ ]:
NAME_OF_DATASET = "i4gi-tjb9"

client = Socrata(
    "data.cityofnewyork.us",
    app_token,
    timeout=60
)

Zunächst laden wir die Meta-Daten herunter.

In [ ]:
metadata = client.get_metadata(NAME_OF_DATASET)

metadata

Hiermit wird nun die PDF mit den weiterführenden Informationen heruntergeladen.
Der Pfad befindet sich in der Ausgabe.

In [ ]:
client.download_attachments(
    NAME_OF_DATASET,
    content_type="json",
    download_dir="."
)

Nun können wir uns die ersten 5000 Datenpunkte anschauen.

In [ ]:
results = client.get(NAME_OF_DATASET, limit=5000)

results

Wir setzen fest, dass wir die letzten 5000 Datenpunkte erhalten wollen.

In [ ]:
df = pd.DataFrame.from_records(results)
df.head()

In [ ]:
df.info()

In diesem Kontext heißt `object`, dass nur Zeichenketten vorliegen.
`data_as_of` ist dem Namen nach aber keine Zeichenkette, sondern ein Datum.
Somit ist die Repräsentation hier noch nicht ganz optimal.

In [ ]:
df = df.assign(data_as_of__as_date=df["data_as_of"].astype('datetime64[ns]'))
df.head()

Das Attribut `link_id` steht für die Station.
Nun wollen wir erst einmal nur eine Station betrachten, und zwar die mit der link_id `4616215`.
Der Aufruf `df.info()` hat als eine Zeile `link_id 5000 non-null object` zurückgegeben.
Der Begriff `object` steht hier für Text, d. h. wir arbeiten nicht mit Zahlen, sondern Zeichenketten.
Deswegen muss der Wert in Anführungszeichen stehen.

In [ ]:
examined_link_id = "4616215"

df_specific_link_id = df[df.link_id == examined_link_id]
df_specific_link_id.head()

In [ ]:
df_specific_link_id.info()

## Visualisierung der betrachteten Strecke

Zunächst visualisieren wir uns eine spezifische Strecke, die ausgewertet wird.
Dafür wird der Text aus der Tabelle in eine Liste von Listen von Gradzahlen umgerechnet.
`latlon` steht hier für Latitude-Longitude (Längengrad-Breitengrad) Koordinaten.

In [ ]:
latlons = []
coordinate_text = df_specific_link_id.iloc[0]["link_points"]
print("Coordinate text:", coordinate_text)
for latlon in coordinate_text.split(" "):
    lat, lon = latlon.split(",")
    lat, lon = float(lat), float(lon)
    latlons.append((lat, lon))

latlons

Diese kann man nun auf einer Karte darstellen.
Dafür bietet sich `folium` an.

In [ ]:
import folium

m = folium.Map(latlons[0], zoom_start=11)
folium.features.PolyLine(latlons, weight=8).add_to(m)
display(m)

### 2.1 Bereinigen der Daten

Häufig kommt es bei solchen CSV-Dateien zu Sprüngen, also das Einträge nicht streng der zeitlichen Reihenfolge nach sortiert sind.
Dies ist häufig mit Problemen in den technischen Systemen erklärbar.
Ob diese Schritte notwendig sind, hängt von der `link_id` ab!

In [ ]:
print("is monotonic increasing: ", df_specific_link_id.index.is_monotonic_increasing)

In [ ]:
# Zum Sortieren, falls obiger Code 'False' ausgibt
df_specific_link_id = df_specific_link_id.sort_index()
print("is monotonic increasing: ", df_specific_link_id.index.is_monotonic_increasing)

Nun stellt sich die Frage, wie häufig die Daten übermittelt werden.
Mit folgendem Trick bekommt man eine Übersicht:

### Attribut speed

Im Folgenden werden die Werte geplottet, um einen visuellen Eindruck zu erhalten.

In [ ]:
speed_series = df_specific_link_id.set_index("data_as_of__as_date").speed.astype("float")
speed_series

In [ ]:
speed_series.plot(style="-")
plt.show()

# Prognose

Damit wir nun den Verkehr über einen längeren Zeitraum betrachten können, werden wir uns auf einen Knoten beschränken und dort erneut Daten der Stadt New York über die API anfragen.
Eine einfache Dokumentation lässt sich wie immer über den Fragezeichen-Operator abfragen.

In [ ]:
?Socrata.get

Nun möchten wir die abgerufenen Informationen von Vornherein auf das Wesentliche beschränken.

In [ ]:
results = client.get(
    NAME_OF_DATASET,
    select="data_as_of, speed",
    where=f"link_id = '{examined_link_id}' AND data_as_of >= '2021-12-01'",
    limit=(12 * 24 * 30 * 3)
)

results

In [ ]:
df = pd.DataFrame.from_records(results)
df.info()
df.head()

Für eine Weiterverarbeitung sollte nun der Text in die passende Einheit umgewandelt werden, sprich in ein Datum oder in eine Fließkommazahl.

In [ ]:
df["data_as_of"] = df["data_as_of"].astype('datetime64[ns]')
df["speed"] = df["speed"].astype('float')
df.set_index("data_as_of", inplace=True)
df.info()
df.head()

Auch hier kann es wieder sein, dass die Reihenfolge der Daten nicht stimmt.
Manche Funktionen von pandas funktionieren dann nicht.
Deswegen sortieren wir die Daten vor.

In [ ]:
print("is monotonic increasing: ", df.index.is_monotonic_increasing)
df = df.sort_index()
print("is monotonic increasing: ", df.index.is_monotonic_increasing)

Nun wollen wir die Geschwindigkeit von gestern betrachten.

In [ ]:
import datetime

today = datetime.datetime.now()

yesterday = today - datetime.timedelta(days=1)

yesterday_as_date = yesterday.date()

yesterday_as_date.isoformat()

In [ ]:
df.speed[yesterday_as_date.isoformat()].plot()
plt.show()

# 2.3 Weiterführende Auswertungen

Für sich wiederholende Muster gibt es verschiedene Möglichkeiten, diese zu visualisieren.
Jede Methode hat ihre eigenen Stärken und Schwächen.

In [ ]:
df = df.loc["2021-11":"2021-12"]
df

### Plots mit Gruppierung

Anstelle der strikten zeitlichen Abfolge können Plots auch anhand verschiedener anderer Kriterien gruppiert und dann ausgewertet werden.
Dies geschieht in pandas mit der Methode `groupby`.

Nun wird die durchschnittliche Geschwindigkeit für jede Stunde des Tages geplottet, also von 00:00 bis 23:00.

In [ ]:
def extract_hour(pandas_timestamp):
    return pandas_timestamp.hour


hourly_speed_mean = df.speed.groupby(extract_hour).agg('mean')
hourly_speed_mean.plot()
plt.show()

(a) Interpretieren Sie den Plot

Nun wird die Standardabweichung dargestellt.
(b) Wie kann man diese nun interpretieren?

In [ ]:
hourly_speed_std = df.speed.groupby(extract_hour).agg('std')
hourly_speed_std.plot()
plt.show()

## 2.5 Autokorrelation

Die Autokorrelation gibt Aufschluss über sich wiederholende Muster.
Wie genau kann man die Abbildung interpretieren?
Wie vielen Minuten entspricht der Lag von 1? 
Wie groß ist die erwartete Zeitspanne für die Autokorrelation? 
Wie groß ist in diesem Beispiel der Lag, den man aus der Alltagserfahrung heraus erwarten würde?

In [ ]:
autocorrelation_plot(df.speed, alpha=0.7)
plt.xlim([0, 5000])  # Begrenze Plot auf die ersten 5000 Einträge
plt.show()

## Prognose der zweiten Monatshälfte anhand der ersten

Wie lässt sich die zweite Monatshälfte mithilfe der ersten Monatshälfte vorhersagen?
Vermutlich lassen sich hier die Prinzipien des vorherigen Notebooks zum Thema Gezeiten anwenden.

In [ ]:
n = len(df)
df_train, df_test = df.iloc[:(n // 2)], df.iloc[(n // 2):]
df_train.info()
df_test.info()
df_train

### 2.6.1 Mittelwert als Baseline

Eine sehr einfache Annahme ist, dass ein Zufallsprozess vorliegt, der der Gauß'schen Normalverteilung folgt.
Alle Ausschläge werden somit als zufällig angesehen.
Für Normalverteilungen ist der Mittelwert ein guter Schätzer ist.

In [ ]:
mean_of_train = df_train.speed.mean()
average_as_estimator = (mean_of_train - df_test.speed)
rmse_with_average = (average_as_estimator ** 2).mean() ** .5
mean_of_train, rmse_with_average

In [ ]:
average_as_estimator.plot()
plt.show()

Jedes Verfahren, das auch nur etwas komplexer als die Mittelwertberechnung ist und dabei höhere Fehler produziert, sollte nur dann verwendet werden, wenn es dafür sehr gute Gründe gibt.
Allgemein gilt, das einfache Mittel bevorzugt werden sollen.

### 2.6.2 Naiver Schätzer


Der nun vorgestellte naive Schätzer funktioniert vergleichbar mit dem naiven Schätzer der Gezeiten in Abschnitt 1.4.2, es wird wieder nur ein Wert aus der Vergangenhet genommen um die Zukunft vorherzusagen. Diesmal liegt jedoch nicht für jeden Zeitpunkt ein Eintrag vor.
Deswegen wird eine Hilfsmethode verwendet, die den nächstgelegenen Eintrag zurückgibt.
Bei den Gezeiten haben wir einen Zyklus von 12h und 25Minuten gewählt. Wie lange ist ein Zyklus in diesem Fall?

(a) Tragen Sie einen Wert ein, der sich aus der Domäne heraus rechtfertigen lässt und begründen Sie Ihre Entscheidung.

In [ ]:
from traffic_time_series_utils import get_nearest_entry

zyklus_laenge_in_stunden = 1  # Hier müssen Sie die Zykluslänge eintragen


def naive_estimator(this_series, t):
    if t < first_day:
        print("Nur für Prognose, nicht in die Vergangenheit")
        return
    nearest_entry = get_nearest_entry(this_series, t)
    if nearest_entry is not None:
        return nearest_entry  # Gebe tatsächlichen Wert zurück
    else:
        t = t - pd.Timedelta(hours=zyklus_laenge_in_stunden)  # Betrachte Wert des vorherigen Zyklus
        return naive_estimator(this_series, t)

Definieren Sie `first_day` und `last_day` anhand des DataFrames `df_train`, wie es bereits bei den Gezeiten vorgenommen worden ist.

(b) Wie bekommen Sie das Datum des ersten Eintrages von df_train?

(c) Wie bekommen Sie das Datum des letzten Eintrages von df_train?

In [ ]:
first_day = ...  # Tragen Sie hier den ersten Tag ein
last_day = ...  # Tragen Sie hier den letzten Tag ein

first_day, last_day

(d) Benutzen Sie nun die Schätzfunktion, um für die zweite Hälfte des Monats den Verkehr zu schätzen.
Hier lohnt sich ebenfalls der Blick in das vorherige Notebook.

In [ ]:
estimated = ...
estimated_as_series = ...
estimated_as_series

Plotten Sie nun das Ergebnis der Schätzung gemeinsam mit den tatsächlichen Daten:

In [ ]:
estimated_as_series.plot()
df_test.speed.plot()
plt.show()

(e) Plotten Sie nun den Fehler und ermitteln Sie die Fehler-Statistiken mittels `describe`.
Wird die Geschwindigkeit eher über- oder unterschätzt?

In [ ]:
# Ihr Code...

(f) Berechnen Sie den Durchschnitt und den RMSE und geben Sie diese an

In [ ]:
# Ihr Code...

(g) Ist nun der naive Schätzer oder der Durchschnitt der bessere Schätzer? Woran könnte dies es liegen? Begründen Sie.

In [ ]:
# Ihr Code...

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0; display:inline" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a> &nbsp;&nbsp;&nbsp;&nbsp;Dieses Werk von Marvin Kastner ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Namensnennung 4.0 International Lizenz</a>.